In [40]:
# %reset
# 'http://www.bseindia.com/download/BhavCopy/Equity/EQ230110_CSV.ZIP'
from datetime import datetime
import pandas as pd
from io import BytesIO
from zipfile import ZipFile
import os
import numpy as np
import pickle
# def __init__(self,)
# def __init__(self, optimizer, loss, validation_data=None):
#         self.optimizer = optimizer
#         self.layers = []
#         self.errors = {"training": [], "validation": []}
#         self.loss_function = loss()
#         self.progressbar = progressbar.ProgressBar(widgets=bar_widgets)

data_folder="D:/SoftwareWebApps/Python/python_wb/Stocks/Data/NSE/"
dirlist=os.listdir(data_folder) # returns list
pickle_path='D:\SoftwareWebApps\Python\python_wb\Stocks\Data\stock_pickle\\'

In [41]:
def save_pkl(file,var):
    with open(file, 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(var, f)
        f.close()
def load_pkl(file):
    with open(file, 'rb') as f:  # Python 3: open(..., 'wb')
        var = pickle.load(f)
        f.close()
    return var

def stock_array(table_name):
    empty_array=[]
    return empty_array

def put_entry(table_arrays,symbols, valarray):
    i=0
    default=[]
    for val in valarray:
        table_arrays.setdefault(symbols[i],default)
#         print(table_arrays[symbols[i]],end=': ')
        table_arrays[symbols[i]].append(val)
#         print(val,end=': ')
#         print(table_arrays[symbols[i]])
        i +=1
    return table_arrays
def armarray_table_fns(symbols):
    return    dict((s,stock_array(s)) for s in symbols)
def update_tables(table_arrays,symbols):
#     print("{} new arrays being created...  ".format(len(symbols)))
    table_arrays.update(armarray_table_fns(symbols))
    return table_arrays

def csvdf_process(myZip):
    df=pd.read_csv(myZip)
    if 'Unnamed: 13' in df.columns:
        df=df.drop(['Unnamed: 13'], axis=1)
    tstamp=datetime.strptime(df['TIMESTAMP'].iloc[0],'%d-%b-%Y')
    symbols=df.SYMBOL.values
    df['TIMESTAMP']=tstamp
    df=df[df.columns[2:-1]].drop('PREVCLOSE',1)
    if len(df.columns)==8:
        df['TOTALTRADES']=0
    values=df.values
                 
    return symbols,values

In [46]:
def get_tablearray(files_not_in_pickle):
    # here table_fns are table arrays
    k=0 #one if tests for k value be careful
    started=datetime.now()
    old=datetime.now()
    table_fns={}
    print("Reading from zip -> csv from {} files".format(len(files_not_in_pickle))
    for file in files_not_in_pickle:
        zip_ref = ZipFile(data_folder+file, 'r')
        try:   
            with zip_ref as my_zip_file:
                with my_zip_file.open(my_zip_file.namelist()[0]) as myZip:
                    symbols,values=csvdf_process(myZip)
                    tables=np.asarray(list(table_fns.keys())).astype(object)        
                    tbls_not_in_db=np.setdiff1d(symbols,tables)            
                    if(k==0):
                        table_fns= update_tables(table_fns,symbols)

                    if(len(tables)>0 and len(tbls_not_in_db)>0):
                        table_fns= update_tables(table_fns,tbls_not_in_db)
                    table_fns=put_entry(table_fns, symbols, values)
                    k+=1
                    print(k,end=', ')
                    if (k%100==0):            
                        print("{} of {} files written to DB..".format(k,len(dirlist)))
                        nowtime=datetime.now()
                        print("last 100files took ..")
                        print(nowtime-old)
                        print("From first it took ..",end='')
                        print(nowtime-started)
                        old=nowtime

        except:
            print("Zip file not good for file: {}".format(file))
    print("Reading from zip -> csv Completed ..")
    return tables, table_fns

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 

In [48]:

DL=pd.DataFrame(dirlist)
# tstamp=pd.Series;
tstam=[]
i=0
for fname in dirlist:
    tstam.append(datetime.strptime(fname[2:11],'%d%b%Y'))
    
DL['tstamp']=tstam   
DL=DL.sort_values(by=['tstamp'])
dirlist=DL[0].tolist()
try:
    columns=load_pkl(pickle_path+'columns_ref.pkl')
    dirlist_written=load_pkl(pickle_path+'wrtitten_files.pkl')
    symbols_pkl=load_pkl(pickle_path+'symbols.pkl')
except:
    columns=[]
    dirlist_written=[]
    symbols_pkl=[]
stime=datetime.now()
files_not_in_pickle=np.setdiff1d(dirlist,dirlist_written)
# files_not_in_pickle=np.intersect1d(dirlist,dirlist_written)
files_not_in_pickle

# tables=np.asarray(list(table_fns.keys())).astype(object) 
tables, table_arrays=get_tablearray(files_not_in_pickle)
tbls_not_in_pkl=np.setdiff1d(tables,symbols_pkl) 
tbls_in_pkl=np.intersect1d(tables,symbols_pkl) 
tbls_all=np.union1d(tables,symbols_pkl) 
# tbls_in_pkl
# len(tbls_all),len(tbls_not_in_pkl),len(tbls_in_pkl)
columns=['OPEN','HIGH','LOW','CLOSE','LAST','TOTTRDQTY','TOTTRDVAL','TIMESTAMP','TOTALTRADES']
stime=datetime.now()
save_pkl(pickle_path+'columns_ref.pkl',columns)
save_pkl(pickle_path+'wrtitten_files.pkl',dirlist)
save_pkl(pickle_path+'symbols.pkl',tbls_all)
save_pkl(self.pickle_path+'latest_symbols.pkl',tables)
i=0
for tbl in tbls_in_pkl:
    # Saving the objects:
    stock_array=load_pkl(pickle_path+tbl+'.pkl')
#     basedf=pd.DataFrame(stock_array,columns=['OPEN','HIGH','LOW','CLOSE','LAST','TOTTRDQTY','TOTTRDVAL','TIMESTAMP','TOTALTRADES'])
#     print(len(stock_array))
    stock_array.extend(table_arrays.get(tbl))    
    save_pkl(pickle_path+tbl+'.pkl',stock_array)
    i +=1
    if i%100 ==0:
        print('{} files among {} re-written'.format(i,len(tbls_in_pkl)))
i=0    
for tbl in tbls_not_in_pkl:
    save_pkl(pickle_path+tbl+'.pkl',table_arrays.get(tbl))
    i +=1
    if i%2 ==0:
        print('{} files among {} written'.format(i,len(tbls_not_in_pkl)))
print("For all It took {} time".format(datetime.now()-stime))
# stock_array
# moddf=pd.DataFrame(stock_array,columns=['OPEN','HIGH','LOW','CLOSE','LAST','TOTTRDQTY','TOTTRDVAL','TIMESTAMP','TOTALTRADES'])
# basedf.tail(6),moddf.tail(6)

100 files among 48 re-written
200 files among 48 re-written
300 files among 48 re-written
400 files among 48 re-written
500 files among 48 re-written
600 files among 48 re-written
700 files among 48 re-written
800 files among 48 re-written
900 files among 48 re-written
1000 files among 48 re-written
1100 files among 48 re-written
1200 files among 48 re-written
1300 files among 48 re-written
1400 files among 48 re-written
1500 files among 48 re-written
1600 files among 48 re-written
1700 files among 48 re-written
1800 files among 48 re-written
2 files among 48 written
4 files among 48 written
6 files among 48 written
8 files among 48 written
10 files among 48 written
12 files among 48 written
14 files among 48 written
16 files among 48 written
18 files among 48 written
20 files among 48 written
22 files among 48 written
24 files among 48 written
26 files among 48 written
28 files among 48 written
30 files among 48 written
32 files among 48 written
34 files among 48 written
36 files amon

In [50]:
# dirlist.sort()
# dirlist

# import os
# from datetime import datetime
# for filename in dirlist:
#     info = os.stat("D:/SoftwareWebApps/Python/python_wb/Stocks/Data/"+filename)
#     print (datetime.fromtimestamp(info.st_mtime))
    
load_pkl(pickle_path+'wrtitten_files.pkl')

['cm03NOV1994bhav.csv.zip',
 'cm07NOV1994bhav.csv.zip',
 'cm08NOV1994bhav.csv.zip',
 'cm09NOV1994bhav.csv.zip',
 'cm10NOV1994bhav.csv.zip',
 'cm11NOV1994bhav.csv.zip',
 'cm14NOV1994bhav.csv.zip',
 'cm15NOV1994bhav.csv.zip',
 'cm16NOV1994bhav.csv.zip',
 'cm17NOV1994bhav.csv.zip',
 'cm21NOV1994bhav.csv.zip',
 'cm22NOV1994bhav.csv.zip',
 'cm23NOV1994bhav.csv.zip',
 'cm24NOV1994bhav.csv.zip',
 'cm25NOV1994bhav.csv.zip',
 'cm28NOV1994bhav.csv.zip',
 'cm29NOV1994bhav.csv.zip',
 'cm30NOV1994bhav.csv.zip',
 'cm01DEC1994bhav.csv.zip',
 'cm02DEC1994bhav.csv.zip',
 'cm05DEC1994bhav.csv.zip',
 'cm06DEC1994bhav.csv.zip',
 'cm07DEC1994bhav.csv.zip',
 'cm08DEC1994bhav.csv.zip',
 'cm09DEC1994bhav.csv.zip',
 'cm12DEC1994bhav.csv.zip',
 'cm13DEC1994bhav.csv.zip',
 'cm14DEC1994bhav.csv.zip',
 'cm15DEC1994bhav.csv.zip',
 'cm16DEC1994bhav.csv.zip',
 'cm19DEC1994bhav.csv.zip',
 'cm20DEC1994bhav.csv.zip',
 'cm21DEC1994bhav.csv.zip',
 'cm22DEC1994bhav.csv.zip',
 'cm23DEC1994bhav.csv.zip',
 'cm26DEC1994bhav.cs